<a href="https://colab.research.google.com/github/KendallScott/Capstone_alt/blob/main/Data_Cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
https://geemap.org/notebooks/114_dynamic_world/


# Imports and Setup

In [2]:
!pip -q install --upgrade folium
!pip -q install geopandas
!pip -q install geojson
!pip -q install eeconvert

In [3]:
# reminder that if you are installing libraries in a Google Colab instance you will be prompted to restart your kernal

try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

package not found, installing w/ pip in Google Colab...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 80.9 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11125 sha256=874aa311c9df5ac4b82ab0378133f811993dd07bbc2c9aceb41205a356940020
  Stored in directory: /root/.c

In [4]:
pip install eemont

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eemont: filename=eemont-0.3.6-py3-none-any.whl size=135742 sha256=c2077488e156192099a51aee017957033b8e51912f45141ed238f08b4fad447a
  Stored in directory: /root/.cache/pip/wheels/c2/79/2c/4fed17c3d3b466bbf4fe5872eec11f189147043b01152a4f75
  Created wheel for ee_extra: filename=ee_extra-0.0.15-py3-none-any.whl size=236755 sha256=31cbafb18f09dd1f3fb8660c52c855517ba8e903cba0908d66de0c33a6701592
  Stored in directory: /root/.cache/pip/wheels/29/96/0e/4e36b0dfd85e16867723df739294c0aa45a65b191adac4d959
Successfully built eemont ee_extra


In [5]:
# Standard imports
import requests
import json

import pandas as pd
import numpy as np
from PIL import Image

# Geospatial processing packages
import geopandas as gpd
import geojson

import shapely
from shapely.geometry import box

# Mapping and plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as cl
import ee
import eeconvert as eec
import geemap
import folium
import eemont

import geopandas as gpd
from shapely.geometry import shape

### Mount Drive

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Authenticate Google Earth Engine
Make sure you have signed up for access to Google Earth Engine at https://signup.earthengine.google.com/#!/. Once your request has been approved, you should be able to access Google Earth Engine at https://code.earthengine.google.com/.

In [7]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=jiSUP255cj6U57FQG39icigFXk2frZekAaxF2SxomP0&tc=iZDl9H6eRujXYPOnt4HJNhOiUCbPA3XVYeCOh4reh00&cc=hnxPO-f9ABoJk3zoi6Q1aHAqRTBquUX2SFeV_CSulgg

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BWxgIX2nSHRvpuZwrCfkiH-Vy86whl8zS8JIbrb_T9NtcUrEQtT5TQ

Successfully saved authorization token.


In [8]:

# Read the GeoJSON file
mines = gpd.read_file('/content/drive/MyDrive/KY_Released_mines.geojson')

# Read data using GeoPandas
print("Data dimensions: {}".format(mines.shape))

Data dimensions: (12606, 19)


In [9]:
mines.sample(10)

,id,ACT_INAC,Calc_Acres,Contact,DATE_ISS,FeatCLS,MINE_STATU,National_I,ORIG_PERM,PERM_ACT,PER_NAME,PER_TYPE,PermitNo,QUAD_DESC,REGION_DES,Shape_Leng,Source,Type_Flag,geometry
1985,00000000000000000558,Released,83.576733,KY Division of Mine Permits,796892400000,SF,RC,KY8070254,8070254,SU 12/06/1995 NW 04/03/1995,GATLIFF COAL COMPANY,PERMANENT,8070254,BALKAN,MIDDLESBORO,12157.939852,KGS Project,TRNS,"POLYGON ((-83.51628 36.78642, -83.51627 36.785..."
8675,0000000000000000129b,Released,72.750886,KY Division of Mine Permits,442483200000,SF,RC,KY8670108,8670108,MI 07/23/1990 MA 09/19/1986 MI ...,C & N COAL COMPANY INC,PERMANENT,8670108,MAYKING,PIKEVILLE,22702.220597,KGS Project,INACT,"POLYGON ((-82.77374 37.17849, -82.77263 37.179..."
8048,00000000000000001249,Released,73.355099,KY Division of Mine Permits,568972800000,SF,RC,KY8668004,8668001,SU 01/12/1988,LEECO INC,PERMANENT,8668004,CUTSHIN,LONDON,11779.370732,KGS Project,INACT,"MULTIPOLYGON (((-83.32923 37.00807, -83.32919 ..."
10631,00000000000000001ec7,Released,42.957189,KY Division of Mine Permits,395910000000,SF,RC,KY0070150,0070150,NW 07/19/1982,BELL COUNTY COAL CORPORATION,INTERIM,0070150,FORK RIDGE,MIDDLESBORO,9547.117362,KGS Project,INACT,"POLYGON ((-83.79519 36.61937, -83.79509 36.618..."
3521,00000000000000002d3e,Released,1.320073,KY Division of Mine Permits,290332800000,SF,RC,KY2365033,2365033,NW 03/15/1979,J C & P COAL CORPORATION,INTERIM,2365033,MCDOWELL,PRESTONSBURG,3019.848783,KGS Project,TRNS,"MULTIPOLYGON (((-82.70460 37.42253, -82.70447 ..."
7592,00000000000000001fa2,Released,28.504973,KY Division of Mine Permits,365065200000,SF,RC,KY0137003,0137003,RV 05/08/1984 NW 07/27/1981,VICTORIA COAL CO,INTERIM,0137003,TIPTOP,LONDON,62341.760222,KGS Project,INACT,"MULTIPOLYGON (((-83.09760 37.55078, -83.09700 ..."
2856,000000000000000030aa,Released,75.640771,KY Division of Mine Permits,477043200000,SF,RC,KY4078007,0078007,SU 10/06/1987 RV 04/29/1986 NW ...,BOW VALLEY COAL (KY) INC,PERMANENT,4078007,BEVERLY,MIDDLESBORO,18713.146536,KGS Project,TRNS,"POLYGON ((-83.59062 36.88561, -83.59049 36.885..."
9109,00000000000000000f3c,Released,5.234381,KY Division of Mine Permits,1268121600000,SF,RC,KY8605336,8605237,SU 2 03/09/2010,LAUREL MOUNTAIN RESOURCES LLC,PERMANENT,8605336,WAYLAND,PRESTONSBURG,1846.904065,KYDMP,INACT,"POLYGON ((-82.83827 37.44924, -82.83803 37.448..."
10815,0000000000000000033f,Released,9.321748,KY Division of Mine Permits,241426800000,SF,RC,KY614677X,614677X,NW 08/26/1977,CASE COAL COMPANY,PRE LAW,614677X,LICK CREEK,PIKEVILLE,8460.591092,KGS Project,INACT,"MULTIPOLYGON (((-82.36710 37.49448, -82.36674 ..."
12220,0000000000000000136c,Released,2.048958,KY Division of Mine Permits,544262400000,SF,RC,KY8674011,8674011,NW 04/01/1987,P S DEVELOPMENT COMPANY,PERMANENT,8674011,JENKINS WEST,PIKEVILLE,1350.161338,KGS Project,INACT,"POLYGON ((-82.73683 37.12632, -82.73653 37.125..."


In [10]:


# Get the shape geometry
region = eec.gdfToFc(mines)

In [11]:
mines_exploded= mines.explode()
mines_exploded

<ipython-input-11-86b5dc24b419>:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  mines_exploded= mines.explode()


id  ACT_INAC  Calc_Acres  \
0     0  00000000000000001111  Released  294.302954   
1     0  00000000000000001112  Released  305.739886   
      1  00000000000000001112  Released  305.739886   
      2  00000000000000001112  Released  305.739886   
      3  00000000000000001112  Released  305.739886   
...                       ...       ...         ...   
12603 2  00000000000000000472  Released   56.660359   
12604 0  0000000000000000308a  Released   14.972560   
      1  0000000000000000308a  Released   14.972560   
      2  0000000000000000308a  Released   14.972560   
12605 0  000000000000000004fc  Released   35.631015   

                             Contact       DATE_ISS FeatCLS MINE_STATU  \
0     0  KY Division of Mine Permits  1141718400000      SF         FF   
1     0  KY Division of Mine Permits  1141718400000      SF         FF   
      1  KY Division of Mine Permits  1141718400000      SF         FF   
      2  KY Division of Mine Permits  1141718400000      SF         FF   
      3  KY Division of Mine Permits  1141718400000      SF         FF   
...                              ...            ...     ...        ...   
12603 2  KY Division of Mine Permits   255254400000      SF         XX   
12604 0  KY Division of Mine Permits   166863600000      SF         XX   
      1  KY Division of Mine Permits   166863600000      SF         XX   
      2  KY Division of Mine Permits   166863600000      SF         XX   
12605 0  KY Division of Mine Permits   262598400000      SF         XX   

        National_I ORIG_PERM  \
0     0  KY8640180   8640180   
1     0  KY8640180   8640180   
      1  KY8640180   8640180   
      2  KY8640180   8640180   
      3  KY8640180   8640180   
...            ...       ...   
12603 2  KY686977X   686977X   
12604 0  KY388574X   388574X   
      1  KY388574X   388574X   
      2  KY388574X   388574X   
12605 0  KY710477X   710477X   

                                                  PERM_ACT  \
0     0  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
1     0  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
      1  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
      2  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
      3  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
...                                                    ...   
12603 2                                 NW      02/02/1978   
12604 0             AM      07/14/1975  NW      04/16/1975   
      1             AM      07/14/1975  NW      04/16/1975   
      2             AM      07/14/1975  NW      04/16/1975   
12605 0                                 NW      04/28/1978   

                        PER_NAME   PER_TYPE PermitNo       QUAD_DESC  \
0     0         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
1     0         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
      1         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
      2         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
      3         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
...                          ...        ...      ...             ...   
12603 2         CAL GLO COAL INC    PRE LAW  686977X    BARBOURVILLE   
12604 0   KY PACIFIC MINING CORP    PRE LAW  388574X    BARBOURVILLE   
      1   KY PACIFIC MINING CORP    PRE LAW  388574X    BARBOURVILLE   
      2   KY PACIFIC MINING CORP    PRE LAW  388574X    BARBOURVILLE   
12605 0  T & T DARBY COAL CO INC    PRE LAW  710477X  PENNINGTON GAP   

           REGION_DES    Shape_Leng       Source Type_Flag  \
0     0  PRESTONSBURG  53267.493940         CHIA      TRNS   
1     0  PRESTONSBURG  92560.008796        KYDMP      TRNS   
      1  PRESTONSBURG  92560.008796        KYDMP      TRNS   
      2  PRESTONSBURG  92560.008796        KYDMP      TRNS   
      3  PRESTONSBURG  92560.008796        KYDMP      TRNS   
...               ...           ...          ...       ...   
12603 2   MIDDLESBORO  16743.038634  KGS Project

In [12]:
# Get the shape geometry for Tennesssee mines
region = eec.gdfToFc(mines)

In [13]:
df = pd.DataFrame()

In [14]:
mines_exploded=mines.explode()
mines_exploded

<ipython-input-14-9f8c0177ef78>:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  mines_exploded=mines.explode()


id  ACT_INAC  Calc_Acres  \
0     0  00000000000000001111  Released  294.302954   
1     0  00000000000000001112  Released  305.739886   
      1  00000000000000001112  Released  305.739886   
      2  00000000000000001112  Released  305.739886   
      3  00000000000000001112  Released  305.739886   
...                       ...       ...         ...   
12603 2  00000000000000000472  Released   56.660359   
12604 0  0000000000000000308a  Released   14.972560   
      1  0000000000000000308a  Released   14.972560   
      2  0000000000000000308a  Released   14.972560   
12605 0  000000000000000004fc  Released   35.631015   

                             Contact       DATE_ISS FeatCLS MINE_STATU  \
0     0  KY Division of Mine Permits  1141718400000      SF         FF   
1     0  KY Division of Mine Permits  1141718400000      SF         FF   
      1  KY Division of Mine Permits  1141718400000      SF         FF   
      2  KY Division of Mine Permits  1141718400000      SF         FF   
      3  KY Division of Mine Permits  1141718400000      SF         FF   
...                              ...            ...     ...        ...   
12603 2  KY Division of Mine Permits   255254400000      SF         XX   
12604 0  KY Division of Mine Permits   166863600000      SF         XX   
      1  KY Division of Mine Permits   166863600000      SF         XX   
      2  KY Division of Mine Permits   166863600000      SF         XX   
12605 0  KY Division of Mine Permits   262598400000      SF         XX   

        National_I ORIG_PERM  \
0     0  KY8640180   8640180   
1     0  KY8640180   8640180   
      1  KY8640180   8640180   
      2  KY8640180   8640180   
      3  KY8640180   8640180   
...            ...       ...   
12603 2  KY686977X   686977X   
12604 0  KY388574X   388574X   
      1  KY388574X   388574X   
      2  KY388574X   388574X   
12605 0  KY710477X   710477X   

                                                  PERM_ACT  \
0     0  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
1     0  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
      1  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
      2  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
      3  SU      03/07/2013  AM   1  05/12/2011  RN   1...   
...                                                    ...   
12603 2                                 NW      02/02/1978   
12604 0             AM      07/14/1975  NW      04/16/1975   
      1             AM      07/14/1975  NW      04/16/1975   
      2             AM      07/14/1975  NW      04/16/1975   
12605 0                                 NW      04/28/1978   

                        PER_NAME   PER_TYPE PermitNo       QUAD_DESC  \
0     0         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
1     0         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
      1         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
      2         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
      3         B & C ENERGY INC  PERMANENT  8640180           MAZIE   
...                          ...        ...      ...             ...   
12603 2         CAL GLO COAL INC    PRE LAW  686977X    BARBOURVILLE   
12604 0   KY PACIFIC MINING CORP    PRE LAW  388574X    BARBOURVILLE   
      1   KY PACIFIC MINING CORP    PRE LAW  388574X    BARBOURVILLE   
      2   KY PACIFIC MINING CORP    PRE LAW  388574X    BARBOURVILLE   
12605 0  T & T DARBY COAL CO INC    PRE LAW  710477X  PENNINGTON GAP   

           REGION_DES    Shape_Leng       Source Type_Flag  \
0     0  PRESTONSBURG  53267.493940         CHIA      TRNS   
1     0  PRESTONSBURG  92560.008796        KYDMP      TRNS   
      1  PRESTONSBURG  92560.008796        KYDMP      TRNS   
      2  PRESTONSBURG  92560.008796        KYDMP      TRNS   
      3  PRESTONSBURG  92560.008796        KYDMP      TRNS   
...               ...           ...          ...       ...   
12603 2   MIDDLESBORO  16743.038634  KGS Project

In [ ]:
df = pd.DataFrame()
years = ['2021-01', '2022-01' ,'2023-01', '2021-07', '2022-07' ,'2023-07']
i=2
z=0

while z<len(years):
  i_date = years[z]+'-01' # Initial date of interest (inclusive)
  f_date = years[z]+'-31' # Final date of interest (exclusive)
  while i<len(mines_exploded):
      fil_mines = mines_exploded.iloc[[i]]
      region = eec.gdfToFc(fil_mines)
      dc = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(region).filterDate(i_date, f_date) # Get dynamic world image collection by asset ID
      polygon = mines_exploded['geometry'].values[i]
      if polygon.geom_type == 'Polygon': # Extract the coordinates from the polygon,
        coordinates = polygon.exterior.coords[:]
        polygon = ee.Geometry.MultiPolygon(coordinates) # Create an ee.Geometry object from the coordinates
        max_error = 1 # Set the maxError parameter
        projected_polygon = polygon.transform('EPSG:4326', max_error) # Project the geometry coordinates

      elif polygon.geom_type == 'LineString':
        coordinates = polygon.coords[:]
        polygon = ee.Geometry.LineString(coordinates)

      df_contructor = []
      ac = dc.getRegion(projected_polygon, scale=200 ).getInfo() #decrease scale to get more rows of data/granularity
      df_constructor =pd.DataFrame(ac)
      df_constructor.columns = df_constructor.iloc[0]
      df_constructor = df_constructor.tail(-1)
      df_constructor['Company'] = mines_exploded['PER_NAME'].values[i]
      df = df.append(df_constructor)

  print(mines_exploded['PER_NAME'].values[i])
  i+=1

  df =df.drop_duplicates()
  df = df[df['label'].notna()]

  file_name = 'ky_'+i_date+f_date
  from google.colab import files
  df.to_csv(file_name, encoding = 'utf-8-sig')
  files.download(file_name)
  z+=1

<ipython-input-16-756b25454b3f>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)
<ipython-input-16-756b25454b3f>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)
<ipython-input-16-756b25454b3f>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)
<ipython-input-16-756b25454b3f>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)
<ipython-input-16-756b25454b3f>:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_con

In [ ]:
i = 0
polygon = mines_exploded['geometry'].values[i]

In [ ]:
# Initial date of interest (inclusive).
i_date = '2021-01-01'
# Final date of interest (exclusive).
f_date = '2021-01-15'

dc = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(region).filterDate(i_date, f_date)


In [ ]:
coordinates.bounds()

In [ ]:
pip install topojson

In [ ]:
import topojson as tp
topo = tp.Topology(mines_exploded, prequantize=False)
mines_exploded = topo.toposimplify(2).to_gdf()

In [ ]:
i=1

# Initial date of interest (inclusive).
i_date = '2021-01-01'
# Final date of interest (exclusive).
f_date = '2021-01-15'

# Extract the coordinates from the polygon
polygon = mines_exploded['geometry'].values[i]

#needs adjusting for different geometry types
coordinates = polygon.exterior.coords[:]
projected_polygon_shape = np.asarray(coordinates)

# Create an ee.Geometry object from the coordinates
polygon = ee.Geometry.MultiPolygon(coordinates)

# Get dynamic world image collection by asset ID
dc = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(i_date, f_date)

# get band names, equal to column namesbandnames = dc.first().bandNames().getInfo()[:-1]

# Set the maxError parameter
max_error = 1  # Specify your desired maxError value

# Project the geometry coordinates
projected_polygon = polygon.transform('EPSG:4326', max_error)

#decrease scale to get more rows of data/granularity
ac = dc.getRegion(projected_polygon, scale=50000000000 ).getInfo()
df_constructor =pd.DataFrame(ac)

df_constructor.columns = df_constructor.iloc[0]
df_constructor = df_constructor.tail(-1)
df_constructor['company'] = mines_exploded['company'].values[i]
df_constructor['shapeName'] = mines_exploded['shapeName'].values[i]
df_constructor['start_date'] = i_date
df_constructor['end_Date'] = f_date
df = df.append(df_constructor)


In [ ]:
i=0

# Initial date of interest (inclusive).
i_date = '2021-01-01'
# Final date of interest (exclusive).
f_date = '2021-01-15'

df_contructor = []
while i<=len(mines_exploded):
 try:
    # Extract the coordinates from the polygon
    polygon = gdf_simplified['geometry'].values[i]

    #needs adjusting for different geometry types
    coordinates = polygon.exterior.coords[:]
    projected_polygon_shape = np.asarray(coordinates)

    projected_polygon_shape = projected_polygon_shape[z]
    bound_object = ee.Geometry.MultiPolygon(coordinates)
    bound_object = bound_object.bounds()

    # Create an ee.Geometry object from the coordinates
    polygon = ee.Geometry.MultiPolygon(coordinates)

    # Get dynamic world image collection by asset ID
    dc = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(i_date, f_date)

    # get band names, equal to column namesbandnames = dc.first().bandNames().getInfo()[:-1]

    # Set the maxError parameter
    max_error = 1  # Specify your desired maxError value

    # Project the geometry coordinates
    projected_polygon = polygon.transform('EPSG:4326', max_error)

    #decrease scale to get more rows of data/granularity
    ac = dc.getRegion(projected_polygon, scale=50000000000 ).getInfo()
    df_constructor =pd.DataFrame(ac)

    df_constructor.columns = df_constructor.iloc[0]
    df_constructor = df_constructor.tail(-1)
    df_constructor['company'] = mines_exploded['company'].values[i]
    df_constructor['shapeName'] = mines_exploded['shapeName'].values[i]
    df_constructor['start_date'] = i_date
    df_constructor['end_Date'] = f_date
    df = df.append(df_constructor)


        print(mines_exploded['company'].values[i])
        i+=1
 except:
          i+=1
          pass



In [ ]:
df['company'].unique()

In [ ]:
df

In [ ]:
len(df['company'].unique())

In [ ]:
len(mines_exploded['company'].unique())